# Chunk-Größe für multilingual-e5-small bestimmen

Das Modell hat eine **maximale Token-Länge von 512 Tokens**.

## Installation und Import

In [1]:
!pip install transformers torch


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
from transformers import AutoTokenizer

# Lade den Tokenizer für multilingual-e5-small
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
print("Tokenizer geladen!")
print(f"Max. Model-Token-Länge: {tokenizer.model_max_length}")

/Users/alice/Repositorys/llm-project-gregus-karbassi/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tokenizer geladen!
Max. Model-Token-Länge: 512


## Test mit Beispieltext aus deinem PDF

In [4]:
# Teste mit unterschiedlich langen Texten
test_texts = {
    "100 Wörter": " ".join(["Wort"] * 100),
    "200 Wörter": " ".join(["Wort"] * 200),
    "300 Wörter": " ".join(["Wort"] * 300),
    "400 Wörter": " ".join(["Wort"] * 400),
}

print("Token-Anzahl für verschiedene Textlängen:\n")
for name, text in test_texts.items():
    # Wichtig: Prefix "passage: " hinzufügen (wird vom Modell erwartet!)
    text_with_prefix = f"passage: {text}"
    tokens = tokenizer(text_with_prefix, return_tensors='pt')
    num_tokens = tokens['input_ids'].shape[1]
    print(f"{name}: {num_tokens} Tokens")
    
print(f"\n⚠️ Maximale Token-Länge: 512 Tokens")
print("Texte werden automatisch abgeschnitten, wenn sie länger sind!")

Token-Anzahl für verschiedene Textlängen:

100 Wörter: 104 Tokens
200 Wörter: 204 Tokens
300 Wörter: 304 Tokens
400 Wörter: 404 Tokens

⚠️ Maximale Token-Länge: 512 Tokens
Texte werden automatisch abgeschnitten, wenn sie länger sind!


## Test mit echtem Text aus deinem Geschichtsbuch

In [5]:
# Lade einen Teil deines extrahierten Textes
with open("docs/extracted_content_geschichtsbuch.md", "r", encoding="utf-8") as f:
    full_text = f.read()

# Teste verschiedene Chunk-Größen (in Zeichen)
chunk_sizes = [500, 1000, 1500, 2000, 2500, 3000]

print("Empfohlene Chunk-Größen (in Zeichen):\n")
print("-" * 60)

for chunk_size in chunk_sizes:
    # Nimm einen Chunk dieser Größe
    sample_chunk = full_text[:chunk_size]
    
    # Tokenisiere mit Prefix
    text_with_prefix = f"passage: {sample_chunk}"
    tokens = tokenizer(text_with_prefix, return_tensors='pt', truncation=False)
    num_tokens = tokens['input_ids'].shape[1]
    
    status = "✅ OK" if num_tokens <= 512 else f"❌ ZU LANG ({num_tokens - 512} Tokens zu viel)"
    
    print(f"Chunk: {chunk_size:4d} Zeichen → {num_tokens:3d} Tokens | {status}")

print("\n" + "=" * 60)
print("💡 EMPFEHLUNG:")
print("=" * 60)

Token indices sequence length is longer than the specified maximum sequence length for this model (651 > 512). Running this sequence through the model will result in indexing errors


Empfohlene Chunk-Größen (in Zeichen):

------------------------------------------------------------
Chunk:  500 Zeichen → 123 Tokens | ✅ OK
Chunk: 1000 Zeichen → 241 Tokens | ✅ OK
Chunk: 1500 Zeichen → 339 Tokens | ✅ OK
Chunk: 2000 Zeichen → 445 Tokens | ✅ OK
Chunk: 2500 Zeichen → 651 Tokens | ❌ ZU LANG (139 Tokens zu viel)
Chunk: 3000 Zeichen → 787 Tokens | ❌ ZU LANG (275 Tokens zu viel)

💡 EMPFEHLUNG:


## Optimale Chunk-Größe berechnen

In [6]:
# Finde die optimale Chunk-Größe durch binäre Suche
def find_optimal_chunk_size(text_sample, max_tokens=510):
    """
    Findet die maximale Chunk-Größe in Zeichen, die nicht über max_tokens kommt.
    510 statt 512, um etwas Puffer zu lassen.
    """
    low, high = 100, 5000
    best_size = 100
    
    while low <= high:
        mid = (low + high) // 2
        chunk = text_sample[:mid]
        text_with_prefix = f"passage: {chunk}"
        tokens = tokenizer(text_with_prefix, return_tensors='pt', truncation=False)
        num_tokens = tokens['input_ids'].shape[1]
        
        if num_tokens <= max_tokens:
            best_size = mid
            low = mid + 1
        else:
            high = mid - 1
    
    return best_size

# Teste mit verschiedenen Textabschnitten
sample1 = full_text[1000:6000]  # Mittelteil
sample2 = full_text[10000:15000]  # Anderer Teil

optimal_size1 = find_optimal_chunk_size(sample1)
optimal_size2 = find_optimal_chunk_size(sample2)

optimal_size = min(optimal_size1, optimal_size2)

print(f"Optimale Chunk-Größe (Probe 1): {optimal_size1} Zeichen")
print(f"Optimale Chunk-Größe (Probe 2): {optimal_size2} Zeichen")
print(f"\n{'='*60}")
print(f"📊 EMPFOHLENE CHUNK-GRÖSSE: {optimal_size} Zeichen")
print(f"{'='*60}")
print(f"\nDas entspricht ungefähr {optimal_size // 5} Wörtern (Durchschnitt)")
print(f"Mit Overlap von 10-20% = {int(optimal_size * 0.1)}-{int(optimal_size * 0.2)} Zeichen")

Optimale Chunk-Größe (Probe 1): 1802 Zeichen
Optimale Chunk-Größe (Probe 2): 2025 Zeichen

📊 EMPFOHLENE CHUNK-GRÖSSE: 1802 Zeichen

Das entspricht ungefähr 360 Wörtern (Durchschnitt)
Mit Overlap von 10-20% = 180-360 Zeichen
